# Skin Cancer: Malignant vs Benign - Part 1 Extracting data

## Introduction
This is a Data Science project , which aim on training a Machine Model from the _Scikit-Learn_ library to detect the two type of skin cancer.

>**Attention**: The use of this work for medical diagnosis is forbidden. If you thing you have a medical issue, please reach out to your family doctor or a specialist.
The author of this project does not take any respinsability for any kind of misuse of the data/results that this project will create.

In this first part, it will be extracted the data form the images, then in hte second, it will be applied the machine learning model.
## Data
The data have been retrieved from the following _Kaggle_ data set, provided by the user _Claudio Fanconi_:
https://www.kaggle.com/fanconic/skin-cancer-malignant-vs-benign

The Data is composed by two different folder:
* Train
* Test

Each one of them with two subfolders of the different type of skin cancer.

Given the fact that the creator of this project is not a medic, it will be provided a definiton for the two types of cancer here below, provided by the website _Medical News Today_ (Link: https://www.medicalnewstoday.com/articles/249141):

* **Benign**: These are not cancerous. They either cannot spread or grow, or they do so very slowly. If a doctor removes them, they do not generally return;
* **Malignant**: Malignant tumors are cancerous. The cells can grow and spread to other parts of the body.

## Strategy
Given the fact that the data are images, first it will be needed to retrive the features of the different images, using a Keras and Tensorflow library, then it will be trained a machine model that will be tested at the end.


## Loading the libraries

In [1]:
# Here will be loaded the different libraries that will be used for the project
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image as Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\admin\anaconda3\envs\exts-ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Creating the generator
datagenerator = ImageDataGenerator(rescale=1/255)

# Creating a graph and session for extracting the features
img_graph = tf.Graph()

with img_graph.as_default():
    # Getting the module
    module_url = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2'
    feature_extractor = hub.Module(module_url)
    
    # Creating input place holder
    input_imgs = tf.placeholder(dtype=tf.float32, shape=[None, 224,224,3])
    
    # Node with features
    imgs_features = feature_extractor(input_imgs)
    
    # Collect initializers
    init_op = tf.group([ tf.global_variables_initializer(), tf.tables_initializer()])
    
    
img_graph.finalize()

# Creating session
sess = tf.Session(graph=img_graph)

# Initialize it
sess.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Loading the data

In [3]:
# Loading the train data
train_data = datagenerator.flow_from_directory('train', batch_size=2637, shuffle=True, target_size=(224,224), color_mode='rgb', class_mode='categorical')

# The train folder contains 2637 images

Found 2637 images belonging to 2 classes.


In [4]:
# Getting the classes
train_data.class_indices

{'benign': 0, 'malignant': 1}

In [5]:
# Loading the test data
test_data = datagenerator.flow_from_directory('test', batch_size=660, shuffle=True, target_size=(224,224), color_mode='rgb', class_mode='categorical')

# The test folder contains 660 images

Found 660 images belonging to 2 classes.


In [6]:
# Getting the classes
test_data.class_indices

{'benign': 0, 'malignant': 1}

## Preparing the data

### Train dataset

In [7]:
batch_train_images, batch_train_labels = train_data.next()

In [8]:
# Converting labels into integer
train_labels = batch_train_labels
train_labels = np.argmax(train_labels,axis=1)

In [9]:
# Extracting the features
train_features = sess.run(imgs_features, feed_dict={input_imgs:batch_train_images})

In [10]:
# Saving the features and labels in an npz file
np.savez('train_data',features=train_features,image=batch_train_images,label=train_labels)

### Test dataset

In [11]:
batch_test_images, batch_test_labels = test_data.next()

In [12]:
# Converting labels into integer
test_labels = batch_test_labels
test_labels = np.argmax(test_labels,axis=1)

In [13]:
# Extracting features
test_features = sess.run(imgs_features, feed_dict={input_imgs: batch_test_images})

In [14]:
# Saving the features and labels in an npz file
np.savez('test_data',features=test_features,image=batch_test_images,label=test_labels)